#把2C基因选出来
import pandas as pd
df=pd.read_csv('ICM_vs_E2C.csv')
newdf=df.loc[(df['pvalue']<0.05) & (df['log2FoldChange']>1)]
newdf.to_csv('2Cgene.csv')

In [116]:
import csv

with open('gencode.vM17.annotation.gff3', 'r') as f:
    lines1 = f.readlines()
with open('high_confidence_enhancer_set.gff3', 'r') as f2:
    lines2 = f2.readlines()
f3 = open('2Cgene.csv','r')
reader = csv.reader(f3) 
f4 = open('2Cgene_within1Mbp_enh.csv','w',newline='')
writer = csv.writer(f4)


In [117]:
#从gencode.vM17.annotation.gff3 中找到genename
def grep_gene(gene_name,filelines):
    for k1 in filelines:
        if '#' in k1 : continue
        k1s=k1.split()
        if k1s[2] == 'gene':
            c1=k1s[8].split(';')
            c2=c1[3].split('=')
            c2=c2[1]
            if c2 == gene_name:
                return k1
            else:continue
        else:continue  


In [118]:
#给定基因位置，找落在其上下游10000000bp的enhancer
def find_enhancer(chrg,geneloc1,geneloc2,filelines):
    enhancerlist=[]
    for enh in filelines:
        enhs = enh.split()
        if enhs[0]==chrg and enhs[2]=='gene' and int(enhs[3])>=int(geneloc1)-1000000 and int(enhs[4])<=int(geneloc2)+1000000 :
            enhancerlist.append(enh)
    return enhancerlist

In [119]:
writer.writerow(['gene_name','chr','GeneId','geneloc1','geneloc2','EnhancerId','enhloc1','enhloc2','r','glog2foldchange','enh_2C/ICM','gene_TPM','enh_TPM'])

113

In [120]:
import numpy as np
with open('Enhancer_TPM_Corrected.txt', 'r') as f5:
    linese = f5.readlines()
with open('gene_TPM_Corrected.txt', 'r') as f6:
    linesg = f6.readlines()

In [121]:
#抓取给定enhancer和gene的TPM，计算correlation,enhancer的foldchange,进行一定的筛选，返回
def TPM_r(enhid,geneid,linese,linesg):
    find_e=0
    fine_g=0
    for tpm_ei in linese:
        tpm_eis=tpm_ei.split()
        if tpm_eis[0] == enhid :
            find_e=1
            enhtpm=(float(tpm_eis[1]), float(tpm_eis[2]), float(tpm_eis[3]), float(tpm_eis[4]), float(tpm_eis[5]), float(tpm_eis[6]), float(tpm_eis[7]),float(tpm_eis[8]), float(tpm_eis[9]))
            break
    for tpm_gi in linesg:
        tpm_gis=tpm_gi.split()
        if tpm_gis[0] == geneid :
            find_g=1
            genetpm=(float(tpm_gis[1]), float(tpm_gis[2]), float(tpm_gis[3]), float(tpm_gis[4]), float(tpm_gis[5]), float(tpm_gis[6]), float(tpm_gis[7]),float(tpm_gis[8]), float(tpm_gis[9]))
            break
    if find_e == 0 | fine_g == 0:
        return -1
    data=pd.DataFrame({'e':enhtpm,'g':genetpm})
    datar=data.corr()
    if(np.isnan(datar.iloc[0,1])) :
        return -1
    else:
        if(float(tpm_eis[2]) == 0):fold=0
        elif float(tpm_eis[6]) == 0 :fold = 10000
        else: fold =float(tpm_eis[2])/float(tpm_eis[6])
    if fold < 1.5 :return -1
    else:return [datar.iloc[0,1],fold,genetpm,enhtpm]       

In [107]:
flag=0
for genei in reader:
    if genei[2] == 'Cdc26':
        break

In [122]:
#对于给定基因csv中的每个基因，找出它的enhancer，算出correlation and foldchange，筛选
for genei in reader:
    if genei[2] == 'gene_name': continue
    c = grep_gene(genei[2],lines1)
    if c == None: continue
    cs=c.split()
    chrg=cs[0]
    geneloc1=cs[3]
    geneloc2=cs[4]
    geneid = cs[8].split(';')
    geneid = geneid[0].split('=')
    geneid = geneid[1]
    elist = find_enhancer(chrg,geneloc1,geneloc2,lines2)
    for enhi in elist:
        enhis=enhi.split()
        enhid=enhis[8].split(';')
        enhid=enhid[0].split('=')
        enhid=enhid[1]
        r = TPM_r(enhid,geneid,linese,linesg)
        if (r == -1) or (abs(r[0])<0.6) :continue
        else:
            writer.writerow([genei[2],chrg,geneid,geneloc1,geneloc2,enhid,enhis[3],enhis[4],r[0],genei[4],r[1],r[2],r[3]])
        

In [103]:
print(genei)

['13087', 'ENSMUSG00000024382.13', 'Ercc3', '582.0631831', '-1.00132544', '0.510852386', '-1.9601072', '0.049983262', '0.086978055', '376.255387', '351.06767', '708.3804323', '943.9940208', '530.6184054']


In [70]:
print(c)

None
